<a href="https://colab.research.google.com/github/tae-su-kim/vllm-101/blob/main/samples/colab_vllm_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vLLM 실습 101

이 노트북에서는 vLLM library의 셋업 및 사용 방법을 간단히 알아봅니다.

## COLAB runtime 설정 (user action required)

이 노트북은 LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct 모델을 vllm library를 이용해서 실행해보도록 작성되었습니다. 이 노트북은 Google Colab에서 제공하는 GPU 환경을 사용하는 것을 전제로 합니다.

먼저 다음과 같이 Colab Runtime을 설정합니다.

메뉴 -> 런타임 -> 런타임 유형 변경 -> T4 GPU (무료 환경 사용 가능).

# 1. vLLM 라이브러리 설치

In [1]:
!pip install vllm==0.7.3

# When running vllm directly from source, use this instead
# %load_ext autoreload
# %autoreload 2
# import sys
# import os
# sys.path.append(os.path.abspath('..'))
## os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

  Using cached vllm-0.7.3-cp38-abi3-manylinux1_x86_64.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully unins

In [ ]:
## Login to huggingface_hub if we need gated model. Not applicable now.
#from huggingface_hub import notebook_login
#notebook_login()

# 2. LLM 모델 로드

In [2]:
from vllm import LLM

model_id = 'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct'
llm = LLM(model=model_id)

INFO 03-25 02:32:18 __init__.py:207] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

INFO 03-25 02:32:19 config.py:2444] Downcasting torch.float32 to torch.float16.
INFO 03-25 02:32:33 config.py:549] This model supports multiple tasks: {'classify', 'generate', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 03-25 02:32:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', speculative_config=None, tokenizer='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-25 02:34:38 model_runner.py:1115] Loading model weights took 4.5145 GB
INFO 03-25 02:34:45 worker.py:267] Memory profiling takes 6.82 seconds
INFO 03-25 02:34:45 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 03-25 02:34:45 worker.py:267] model weights take 4.51GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.95GiB; the rest of the memory reserved for KV Cache is 6.76GiB.
INFO 03-25 02:34:46 executor_base.py:111] # cuda blocks: 5906, # CPU blocks: 3495
INFO 03-25 02:34:46 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 2.88x
INFO 03-25 02:34:55 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [01:03<00:00,  1.80s/it]

INFO 03-25 02:35:58 model_runner.py:1562] Graph capturing finished in 63 secs, took 0.21 GiB
INFO 03-25 02:35:58 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 79.75 seconds


이 셀이 성공적으로 실행되었다면, vLLM 설정과 huggingface 접근이 잘 수행된 것입니다.

추가로 현재 노트북에서 display를 개선하기 위한 코드를 실행합니다 (vLLM과 관련 없음).

In [3]:
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

## 해당 언어 모델을 위한 시스템 프롬프트 준비
이 시스템 프롬프트는 [LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct](https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct) 모델의 repository에서 예시로 사용하고 있는 프롬프트입니다. 프롬프트는 활용에 따라서 자유롭게 수정할 수 있고, 많은 경우에 실제 유저 경험에도 큰 영향을 미치게 됩니다. 자유롭게 변경해보세요!

In [4]:
SYSTEM_PROMPT = """\
You are EXAONE model from LG AI Research, a helpful assistant.\
"""

def get_prompt(message: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    return f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

## Offline inference 테스트

위에서 생성한 vllm.LLM 인스턴스를 통해 LLM 모델의 offline inference를 실험해볼 수 있습니다. Offline inference는 API 서빙을 위한 batching 등 front-end 기능을 제외한 것으로, LLM inference 자체를 실험 해 보고 싶을때, 혹은 로컬 스크립트에서 LLM 기능이 필요할때 사용할 수 있습니다.

[SamplingParams](https://docs.vllm.ai/en/v0.6.0/dev/sampling_params.html)는 llm 모델의 출력 토큰 생성시에 sampling 과정을 정의할 수 있는 변수로, top_p, top_k, temperature 등 여러 파라미터를 수정하여 output을 바꿀 수 있습니다. 해당 옵션들은 서빙 성능에도 영향이 있고, 모델 성능에도 영향이 있으므로 활용처에 따라 다르게 세팅할 필요가 있습니다.

In [7]:
from vllm import SamplingParams

sampling_params = SamplingParams()
sampling_params.max_tokens = 100
sampling_params.top_p = 0.5
sampling_params.top_k = 10
sampling_params.temperature = 0.7

question = "판교역에 대해 알려줘."
prompt = get_prompt(question)

display_header("Prompt:")
display_content(prompt)

display_header("Answer:")
result = llm.generate(prompt, sampling_params=sampling_params)
display_content(result[0].outputs[0].text)


**Prompt:**

```
<s>[INST] <<SYS>>
You are EXAONE model from LG AI Research, a helpful assistant.
<</SYS>>

판교역에 대해 알려줘. [/INST]
```

**Answer:**

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it, est. speed input: 36.14 toks/s, output: 33.73 toks/s]


```

판교역은 경기도 성남시 분당구에 위치한 주요 교통 허브로, 신분당선과 경의중앙선이 교차하며 인근에 다양한 상업 시설과 주거 단지가 있어 접근성이 뛰어납니다.
```

결과가 성공적으로 출력되었다면, vLLM의 기본적인 사용에 성공하셨습니다!

## Online Serving 테스트

Offline Serving에 성공하였으니, 이제 Online Serving API를 띄워봅니다. LLM의 Online Serving은 보통 OpenAI API-compatible한 포맷으로 지원하는 경우가 많으며, vllm에서도 이를 지원하고 있습니다.

In [ ]:
import subprocess
import time
import os

del LLM

# Start vllm server in the background
vllm_process = subprocess.Popen([
    'vllm',
    'serve',  # Subcommand must follow vllm
    model_id,
    '--trust-remote-code',
    '--dtype', 'half',
    '--max-model-len', '2048',
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, start_new_session=True)

# vLLM API 상태 확인하기

서빙 API를 subprocess로 실행하였으므로 해당 프로세스에 직접 HTTP 요청을 통해 상태를 확인합니다.

In [ ]:
import requests

def check_vllm_status():
    try:
        response = requests.get("http://localhost:8000/health")
        if response.status_code == 200:
            print("vllm server is running")
            return True
    except requests.exceptions.ConnectionError:
        print("vllm server is not running")
        return False

try:
    # Monitor the process
    while True:
        if check_vllm_status() == True:
            print("The vllm server is ready to serve.")
            break
        else:
            print("The vllm server has stopped.")
            stdout, stderr = vllm_process.communicate(timeout=10)
            print(f"STDOUT: {stdout.decode('utf-8')}")
            print(f"STDERR: {stderr.decode('utf-8')}")
            break
        time.sleep(5)  # Check every second
except KeyboardInterrupt:
    print("Stopping the check of vllm...")